<h2>Création ou extration des données pour obtenir le fichier csv</h2>

In [3]:
# installation de pandas et requests

In [6]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
import requests
import pandas as pd

API_KEY = "85a23812c5c3bde3c8c179a8da7ad744"
BASE_URL = "http://api.aviationstack.com/v1/flights"

# Paramètres de la requête
params = {
    "access_key" : API_KEY,
    "limit": 100,
    "flight_status": "landed"
}

# Liste pour stocker les vols
all_flights = []

# Récupérer plusieurs pages de données
for offset in range(0, 500, 100):  # 500 vols, par pas de 100
    params["offset"] = offset  # Changer l’offset à chaque itération

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json().get("data", [])
        
        for flight in data:
            all_flights.append([
                flight.get("flight_date"),
                flight.get("flight_status"),
                flight["departure"].get("airport") if flight.get("departure") else None,
                flight["departure"].get("iata") if flight.get("departure") else None,
                flight["departure"].get("delay") if flight.get("departure") else None,
                flight["arrival"].get("airport") if flight.get("arrival") else None,
                flight["arrival"].get("iata") if flight.get("arrival") else None,
                flight["arrival"].get("delay") if flight.get("arrival") else None,
                flight["airline"].get("name") if flight.get("airline") else None,
                flight["flight"].get("number") if flight.get("flight") else None,
                flight["flight"].get("iata") if flight.get("flight") else None
            ])
    
    else:
        print(f"⚠️ Erreur avec l'API (Offset {offset}): {response.status_code}")
        break  # Stopper en cas d'erreur pour éviter de gaspiller des requêtes

# Création du DataFrame Pandas
df = pd.DataFrame(all_flights, columns=[
    "Date", "Statut", "Aeroport_Depart", "Code_IATA_Depart", "Retard_Depart",
    "Aeroport_Arrivee", "Code_IATA_Arrivee", "Retard_Arrivee",
    "Compagnie", "Numero_Vol", "Code_IATA_Vol"
])

# Sauvegarde en CSV
df.to_csv("vols_grand_dataset.csv", index=False, encoding="utf-8")
print(f"Données enregistrées ({len(df)} vols) dans vols_grand_dataset.csv")
    

Données enregistrées (500 vols) dans vols_grand_dataset.csv


 <h2>Charger et vérifier les données 

In [15]:

# Charger le dataset
df = pd.read_csv("vols_grand_dataset.csv")

# Afficher les premières lignes
print(df.head())


# Vérifier les colonnes et valeurs manquantes
print(df.info())

# Vérifier les statistiques des retards
print(df.describe())

         Date  Statut   Aeroport_Depart Code_IATA_Depart  Retard_Depart  \
0  2025-03-15  landed            Broome              BME            NaN   
1  2025-03-15  landed            Hobart              HBA           10.0   
2  2025-03-15  landed  Singapore Changi              SIN           25.0   
3  2025-03-15  landed  Singapore Changi              SIN           14.0   
4  2025-03-15  landed          Dushanbe              DYU            NaN   

                  Aeroport_Arrivee Code_IATA_Arrivee  Retard_Arrivee  \
0                        Kununurra               KNX             NaN   
1  Melbourne - Tullamarine Airport               MEL             NaN   
2     Newark Liberty International               EWR             NaN   
3                            Civil               TRZ             NaN   
4                           Urumqi               URC             NaN   

            Compagnie  Numero_Vol Code_IATA_Vol  
0        Thai AirAsia       608.0         FD608  
1              Q

<h2>Nettoyage et Préparation des Dnnées</h2>

In [17]:
# Supprimer les vols sans Retard_Depart
df_clean = df.dropna(subset=["Retard_Depart"]).copy()

# Remplacer les NaN dans Retard_Arrivee par 0 (sans inplace=True)
df_clean["Retard_Arrivee"] = df_clean["Retard_Arrivee"].fillna(0)

# Convertir les retards en entiers
df_clean["Retard_Depart"] = df_clean["Retard_Depart"].astype(int)
df_clean["Retard_Arrivee"] = df_clean["Retard_Arrivee"].astype(int)

# Convertir les statuts en valeurs numériques
df_clean["Statut"] = df_clean["Statut"].map({
    "scheduled": 0,
    "active": 1,
    "landed": 2,
    "cancelled": -1
}).fillna(0).astype(int)

# Vérification des données après nettoyage
print(df_clean.info())
print(df_clean.head())


<class 'pandas.core.frame.DataFrame'>
Index: 402 entries, 1 to 499
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               402 non-null    object 
 1   Statut             402 non-null    int32  
 2   Aeroport_Depart    402 non-null    object 
 3   Code_IATA_Depart   402 non-null    object 
 4   Retard_Depart      402 non-null    int32  
 5   Aeroport_Arrivee   399 non-null    object 
 6   Code_IATA_Arrivee  402 non-null    object 
 7   Retard_Arrivee     402 non-null    int32  
 8   Compagnie          399 non-null    object 
 9   Numero_Vol         398 non-null    float64
 10  Code_IATA_Vol      398 non-null    object 
dtypes: float64(1), int32(3), object(7)
memory usage: 33.0+ KB
None
         Date  Statut   Aeroport_Depart Code_IATA_Depart  Retard_Depart  \
1  2025-03-15       2            Hobart              HBA             10   
2  2025-03-15       2  Singapore Changi              SIN  

# Entraînement du Modèle Isolation Forest

In [22]:
from sklearn.ensemble import IsolationForest

# Sélection des variables pour l'entraînement
df_ml = df_clean[["Statut", "Retard_Depart", "Retard_Arrivee"]]

# Création et entraînement du modèle Isolation Forest
model = IsolationForest(contamination=0.05, random_state=42)
df_clean["Anomalie"] = model.fit_predict(df_ml)

# Compter les anomalies détectées
anomalies = df_clean[df_clean["Anomalie"] == -1]
print(f"{len(anomalies)} vols suspects détectés sur {len(df_clean)}.")
print(anomalies.head())


21 vols suspects détectés sur 402.
           Date  Statut                 Aeroport_Depart Code_IATA_Depart  \
98   2025-03-15       2  Guangzhou Baiyun International              CAN   
134  2025-03-15       2         Hong Kong International              HKG   
166  2025-03-15       2                         Wenzhou              WNZ   
183  2025-03-15       2         Hong Kong International              HKG   
264  2025-03-15       2      Christchurch International              CHC   

     Retard_Depart                                   Aeroport_Arrivee  \
98              38                                          Don Muang   
134             41                                    Seoul (Incheon)   
166            112                                           Shenzhen   
183             40  Taiwan Taoyuan International (Chiang Kai Shek ...   
264             48                             Auckland International   

    Code_IATA_Arrivee  Retard_Arrivee    Compagnie  Numero_Vol Code_I

In [21]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Normalisation des données
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_ml)

# Entraîner DBSCAN
dbscan = DBSCAN(eps=2, min_samples=3).fit(df_scaled)
df_clean["Anomalie_DBSCAN"] = dbscan.labels_

# Vérifier les anomalies détectées
dbscan_anomalies = df_clean[df_clean["Anomalie_DBSCAN"] == -1]
print(f"{len(dbscan_anomalies)} vols suspects détectés avec DBSCAN sur {len(df_clean)}.")
print(dbscan_anomalies.head())


1 vols suspects détectés avec DBSCAN sur 402.
           Date  Statut Aeroport_Depart Code_IATA_Depart  Retard_Depart  \
166  2025-03-15       2         Wenzhou              WNZ            112   

    Aeroport_Arrivee Code_IATA_Arrivee  Retard_Arrivee    Compagnie  \
166         Shenzhen               SZX              19  SF Airlines   

     Numero_Vol Code_IATA_Vol  Anomalie  Anomalie_DBSCAN  
166      7277.0        O37277        -1               -1  


<h2>Script qui englobe Isolation Forest and DBSCAN </h2>

In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Charger les données de vols
df = pd.read_csv("vols_grand_dataset.csv")

# Nettoyage et prétraitement
df_clean = df.dropna(subset=["Retard_Depart"]).copy()
df_clean["Retard_Arrivee"] = df_clean["Retard_Arrivee"].fillna(0)

df_clean["Retard_Depart"] = df_clean["Retard_Depart"].astype(int)
df_clean["Retard_Arrivee"] = df_clean["Retard_Arrivee"].astype(int)

df_clean["Statut"] = df_clean["Statut"].map({
    "scheduled": 0, "active": 1, "landed": 2, "cancelled": -1
}).fillna(0).astype(int)

# Préparation des variables pour les modèles
df_ml = df_clean[["Statut", "Retard_Depart", "Retard_Arrivee"]]

# Isolation Forest
model_iso = IsolationForest(contamination=0.05, random_state=42)
df_clean["Anomalie_ISO"] = model_iso.fit_predict(df_ml)

# DBSCAN (Normalisation nécessaire)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_ml)

model_dbscan = DBSCAN(eps=2, min_samples=3)
df_clean["Anomalie_DBSCAN"] = model_dbscan.fit_predict(df_scaled)

# Marquer les anomalies finales (vols détectés par les deux modèles)
df_clean["Anomalie_Final"] = np.where(
    (df_clean["Anomalie_ISO"] == -1) & (df_clean["Anomalie_DBSCAN"] == -1), -1, 1
)

# Exporter les résultats dans un CSV
df_clean.to_csv("vols_anomalies.csv", index=False, encoding="utf-8")
print(f"Détection terminée ! {len(df_clean[df_clean['Anomalie_Final'] == -1])} anomalies trouvées.")


Détection terminée ! 1 anomalies trouvées.
